https://www.bilibili.com/video/BV1Xi421R7ev/?spm_id_from=333.337.search-card.all.click&vd_source=a0dab138978e49c265fbce6907b40d53

https://www.bilibili.com/video/BV1F1421B7iv/?spm_id_from=333.337.search-card.all.click&vd_source=a0dab138978e49c265fbce6907b40d53

https://www.youtube.com/watch?v=o29P0Kpobz0

https://www.youtube.com/watch?v=SMBkImDWOyQ

In [2]:
import torch
import math
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# verify the expand function
a = torch.arange(0, 4)
print("arange(0, 4) shape:", a.shape)
print("a:\n", a)

b1 = a.unsqueeze(0)
print("unsqueeze(0) shape:", b1.shape)
print("b1:\n", b1)
b1 = b1[:, None, :]
print("[:, None, :] shape:", b1.shape)
print("b1:\n", b1)

b = a[None, :, None]
print("[None, :, None] shape:", b.shape)
print("b:\n", b)

c = b.expand(4, -1, 1)
print("c.shape:", c.shape)
print("c:\n", c)

b[0][1] = 100
print("b:\n", b)
print("c:\n", c)

In [4]:
def rotate_half(x):
    """Rotates half the hidden dims of the input."""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)

In [92]:
base = 100
dim = 16  
device = 'cpu'
exp = torch.arange(0, dim, 2, dtype=torch.float32).to(device) / dim
print(exp)
# inv_freq = 1.0 / (10000 ** (2i/dim))
# shape: [half_dim]
inv_freq = 1.0 / (base ** exp)

print(inv_freq)
print(inv_freq.shape)

tensor([0.0000, 0.1250, 0.2500, 0.3750, 0.5000, 0.6250, 0.7500, 0.8750])
tensor([1.0000, 0.5623, 0.3162, 0.1778, 0.1000, 0.0562, 0.0316, 0.0178])
torch.Size([8])


In [80]:
context_len = 4
# shape [context_len]
position_ids = torch.arange(0, context_len)
print(position_ids.shape)

# shape [1, context_len]
position_ids = position_ids.unsqueeze(0)
print(position_ids)
print(position_ids.shape)

torch.Size([4])
tensor([[0, 1, 2, 3]])
torch.Size([1, 4])


In [82]:
# [1, half_dim, 1]
inv_freq_expanded = inv_freq[None, :, None].float()
print(inv_freq_expanded)
print(inv_freq_expanded.shape)

# [batch_size, half_dim, 1]
inv_freq_expanded = inv_freq_expanded.expand(position_ids.shape[0], -1, 1)
print(inv_freq_expanded)
print(inv_freq_expanded.shape)

tensor([[[1.0000],
         [0.5623],
         [0.3162],
         [0.1778],
         [0.1000],
         [0.0562],
         [0.0316],
         [0.0178]]])
torch.Size([1, 8, 1])
tensor([[[1.0000],
         [0.5623],
         [0.3162],
         [0.1778],
         [0.1000],
         [0.0562],
         [0.0316],
         [0.0178]]])
torch.Size([1, 8, 1])


In [81]:
# [1, 1, context_len]
position_ids_expanded = position_ids[:, None, :].float()
print(position_ids_expanded)
print(position_ids_expanded.shape)


tensor([[[0., 1., 2., 3.]]])
torch.Size([1, 1, 4])


In [77]:
freqs = (inv_freq_expanded.float() @ position_ids_expanded.float())
print(freqs)
print(freqs.shape)

freqs = freqs.transpose(1, 2)
print(freqs)
print(freqs.shape)

tensor([[[0.0000, 1.0000, 2.0000, 3.0000],
         [0.0000, 0.5623, 1.1247, 1.6870],
         [0.0000, 0.3162, 0.6325, 0.9487],
         [0.0000, 0.1778, 0.3557, 0.5335],
         [0.0000, 0.1000, 0.2000, 0.3000],
         [0.0000, 0.0562, 0.1125, 0.1687],
         [0.0000, 0.0316, 0.0632, 0.0949],
         [0.0000, 0.0178, 0.0356, 0.0533]]])
torch.Size([1, 8, 4])
tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [1.0000, 0.5623, 0.3162, 0.1778, 0.1000, 0.0562, 0.0316, 0.0178],
         [2.0000, 1.1247, 0.6325, 0.3557, 0.2000, 0.1125, 0.0632, 0.0356],
         [3.0000, 1.6870, 0.9487, 0.5335, 0.3000, 0.1687, 0.0949, 0.0533]]])
torch.Size([1, 4, 8])


In [ ]:
# full rotary positinal encoding
base = 100
context_len = 4
dim = 16
device = 'cpu'
exp = torch.arange(0, dim, 2, dtype=torch.float32).to(device) / dim
print(exp)
# inv_freq = 1.0 / (10000 ** (2i/dim))
inv_freq = 1.0 / (base ** exp)
print(inv_freq)

position_ids = torch.arange(0, context_len)
position_ids = position_ids.unsqueeze(0)
print(position_ids)
print(position_ids.shape)

inv_freq_expanded = inv_freq[None, :, None].float()
print(inv_freq_expanded)
print(inv_freq_expanded.shape)
inv_freq_expanded = inv_freq_expanded.expand(position_ids.shape[0], -1, 1)
print(inv_freq_expanded)
print(inv_freq_expanded.shape)

position_ids_expanded = position_ids[:, None, :].float()
print(position_ids_expanded)
print(position_ids_expanded.shape)

freqs = (inv_freq_expanded.float() @ position_ids_expanded.float())
print(freqs)
print(freqs.shape)

freqs = freqs.transpose(1, 2)
print(freqs)
print(freqs.shape)

emb = torch.cat((freqs, freqs), dim=-1)
cos = emb.cos()
sin = emb.sin()

x = torch.randn((context_len, dim))
print("x:\n", x)

y = rotate_half(x)
print("y:\n", y)

out = x * cos + y * sin
print("out:\n", out)


tensor([0.0000, 0.1250, 0.2500, 0.3750, 0.5000, 0.6250, 0.7500, 0.8750])
tensor([1.0000, 0.5623, 0.3162, 0.1778, 0.1000, 0.0562, 0.0316, 0.0178])
tensor([[0, 1, 2, 3]])
torch.Size([1, 4])
tensor([[[1.0000],
         [0.5623],
         [0.3162],
         [0.1778],
         [0.1000],
         [0.0562],
         [0.0316],
         [0.0178]]])
torch.Size([1, 8, 1])
tensor([[[1.0000],
         [0.5623],
         [0.3162],
         [0.1778],
         [0.1000],
         [0.0562],
         [0.0316],
         [0.0178]]])
torch.Size([1, 8, 1])
tensor([[[0., 1., 2., 3.]]])
torch.Size([1, 1, 4])
tensor([[[0.0000, 1.0000, 2.0000, 3.0000],
         [0.0000, 0.5623, 1.1247, 1.6870],
         [0.0000, 0.3162, 0.6325, 0.9487],
         [0.0000, 0.1778, 0.3557, 0.5335],
         [0.0000, 0.1000, 0.2000, 0.3000],
         [0.0000, 0.0562, 0.1125, 0.1687],
         [0.0000, 0.0316, 0.0632, 0.0949],
         [0.0000, 0.0178, 0.0356, 0.0533]]])
torch.Size([1, 8, 4])
tensor([[[0.0000, 0.0000, 0.0000, 0.0000,

In [86]:
# multiple batch
batch_size = 3

# full rotary positinal encoding
base = 100
context_len = 4
dim = 16
device = 'cpu'
exp = torch.arange(0, dim, 2, dtype=torch.float32).to(device) / dim
print(exp)
# inv_freq = 1.0 / (10000 ** (2i/dim))
inv_freq = 1.0 / (base ** exp)
print(inv_freq)

# Generate position IDs for multiple batches
position_ids = torch.arange(context_len).unsqueeze(0).repeat(batch_size, 1)
print(position_ids.shape)
print(position_ids)


tensor([0.0000, 0.1250, 0.2500, 0.3750, 0.5000, 0.6250, 0.7500, 0.8750])
tensor([1.0000, 0.5623, 0.3162, 0.1778, 0.1000, 0.0562, 0.0316, 0.0178])
torch.Size([3, 4])
tensor([[0, 1, 2, 3],
        [0, 1, 2, 3],
        [0, 1, 2, 3]])


In [87]:
inv_freq_expanded = inv_freq[None, :, None].float()
print(inv_freq_expanded)
print(inv_freq_expanded.shape)



tensor([[[1.0000],
         [0.5623],
         [0.3162],
         [0.1778],
         [0.1000],
         [0.0562],
         [0.0316],
         [0.0178]]])
torch.Size([1, 8, 1])


In [88]:
inv_freq_expanded = inv_freq_expanded.expand(position_ids.shape[0], -1, 1)
print(inv_freq_expanded)
print("inv_freq_expanded.shape:", inv_freq_expanded.shape)



tensor([[[1.0000],
         [0.5623],
         [0.3162],
         [0.1778],
         [0.1000],
         [0.0562],
         [0.0316],
         [0.0178]],

        [[1.0000],
         [0.5623],
         [0.3162],
         [0.1778],
         [0.1000],
         [0.0562],
         [0.0316],
         [0.0178]],

        [[1.0000],
         [0.5623],
         [0.3162],
         [0.1778],
         [0.1000],
         [0.0562],
         [0.0316],
         [0.0178]]])
inv_freq_expanded.shape: torch.Size([3, 8, 1])


In [90]:
position_ids_expanded = position_ids[:, None, :].float()
print(position_ids_expanded)
print("inv_freq_expanded.shape:", position_ids_expanded.shape)



tensor([[[0., 1., 2., 3.]],

        [[0., 1., 2., 3.]],

        [[0., 1., 2., 3.]]])
inv_freq_expanded.shape: torch.Size([3, 1, 4])


In [91]:
freqs = (inv_freq_expanded.float() @ position_ids_expanded.float())
print(freqs)
print(freqs.shape)



tensor([[[0.0000, 1.0000, 2.0000, 3.0000],
         [0.0000, 0.5623, 1.1247, 1.6870],
         [0.0000, 0.3162, 0.6325, 0.9487],
         [0.0000, 0.1778, 0.3557, 0.5335],
         [0.0000, 0.1000, 0.2000, 0.3000],
         [0.0000, 0.0562, 0.1125, 0.1687],
         [0.0000, 0.0316, 0.0632, 0.0949],
         [0.0000, 0.0178, 0.0356, 0.0533]],

        [[0.0000, 1.0000, 2.0000, 3.0000],
         [0.0000, 0.5623, 1.1247, 1.6870],
         [0.0000, 0.3162, 0.6325, 0.9487],
         [0.0000, 0.1778, 0.3557, 0.5335],
         [0.0000, 0.1000, 0.2000, 0.3000],
         [0.0000, 0.0562, 0.1125, 0.1687],
         [0.0000, 0.0316, 0.0632, 0.0949],
         [0.0000, 0.0178, 0.0356, 0.0533]],

        [[0.0000, 1.0000, 2.0000, 3.0000],
         [0.0000, 0.5623, 1.1247, 1.6870],
         [0.0000, 0.3162, 0.6325, 0.9487],
         [0.0000, 0.1778, 0.3557, 0.5335],
         [0.0000, 0.1000, 0.2000, 0.3000],
         [0.0000, 0.0562, 0.1125, 0.1687],
         [0.0000, 0.0316, 0.0632, 0.0949],
       

In [85]:
# multiple batch
batch_size = 3

# full rotary positinal encoding
base = 100
context_len = 4
dim = 16
device = 'cpu'
exp = torch.arange(0, dim, 2, dtype=torch.float32).to(device) / dim
print(exp)
# inv_freq = 1.0 / (10000 ** (2i/dim))
inv_freq = 1.0 / (base ** exp)
print(inv_freq)

# Generate position IDs for multiple batches
position_ids = torch.arange(context_len).unsqueeze(0).repeat(batch_size, 1)
print(position_ids.shape)

inv_freq_expanded = inv_freq[None, :, None].float()
# print(inv_freq_expanded)
# print(inv_freq_expanded.shape)
inv_freq_expanded = inv_freq_expanded.expand(position_ids.shape[0], -1, 1)
print(inv_freq_expanded)
print("inv_freq_expanded.shape:", inv_freq_expanded.shape)

position_ids_expanded = position_ids[:, None, :].float()
print(position_ids_expanded)
print("inv_freq_expanded.shape:", inv_freq_expanded.shape)

freqs = (inv_freq_expanded.float() @ position_ids_expanded.float())
print(freqs)
print(freqs.shape)

freqs = freqs.transpose(1, 2)
print(freqs)
print(freqs.shape)

emb = torch.cat((freqs, freqs), dim=-1)
cos = emb.cos()
sin = emb.sin()

x = torch.randn((context_len, dim))
print("x:\n", x)

y = rotate_half(x)
print("y:\n", y)

out = x * cos + y * sin
print("out:\n", out)


tensor([0.0000, 0.1250, 0.2500, 0.3750, 0.5000, 0.6250, 0.7500, 0.8750])
tensor([1.0000, 0.5623, 0.3162, 0.1778, 0.1000, 0.0562, 0.0316, 0.0178])
torch.Size([3, 4])
tensor([[[1.0000],
         [0.5623],
         [0.3162],
         [0.1778],
         [0.1000],
         [0.0562],
         [0.0316],
         [0.0178]],

        [[1.0000],
         [0.5623],
         [0.3162],
         [0.1778],
         [0.1000],
         [0.0562],
         [0.0316],
         [0.0178]],

        [[1.0000],
         [0.5623],
         [0.3162],
         [0.1778],
         [0.1000],
         [0.0562],
         [0.0316],
         [0.0178]]])
inv_freq_expanded.shape: torch.Size([3, 8, 1])
tensor([[[0., 1., 2., 3.]],

        [[0., 1., 2., 3.]],

        [[0., 1., 2., 3.]]])
inv_freq_expanded.shape: torch.Size([3, 8, 1])
tensor([[[0.0000, 1.0000, 2.0000, 3.0000],
         [0.0000, 0.5623, 1.1247, 1.6870],
         [0.0000, 0.3162, 0.6325, 0.9487],
         [0.0000, 0.1778, 0.3557, 0.5335],
         [0.0000, 0.